This notebook provides examples to go along with the [textbook](https://underactuated.csail.mit.edu/contact.html).  I recommend having both windows open, side-by-side!


In [ ]:
import numpy as np
from pydrake.all import (
    DiagramBuilder,
    TrajectorySource,
    MultibodyPlant,
    SceneGraph,
    RobotDiagramBuilder,
    Parser,
    Solve,
    MultibodyPositionToGeometryPose,
    Role,
    MeshcatVisualizer,
    HybridMultibodyCollocation,
    StartMeshcat,
    Simulator,
)

from underactuated import running_as_notebook

np.set_printoptions(suppress=True, precision=6)

In [ ]:
meshcat = StartMeshcat()

# Hybrid Multibody Collocation



In [ ]:
def Animate(xml, hybrid, result):
    x_traj = hybrid.ReconstructStateTrajectory(result)

    builder = DiagramBuilder()
    plant = MultibodyPlant(0.0)
    scene_graph = builder.AddSystem(SceneGraph())
    plant.RegisterAsSourceForSceneGraph(scene_graph)
    Parser(plant, scene_graph).AddModelsFromString(xml, "xml")
    plant.Finalize()

    source = builder.AddSystem(TrajectorySource(x_traj))
    pos_to_pose = builder.AddSystem(
        MultibodyPositionToGeometryPose(plant, input_multibody_state=True)
    )
    builder.Connect(source.get_output_port(0), pos_to_pose.get_input_port())
    builder.Connect(
        pos_to_pose.get_output_port(),
        scene_graph.get_source_pose_port(plant.get_source_id()),
    )

    meshcat.Set2dRenderMode(xmin=-5, xmax=5, ymin=-2, ymax=5)
    visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)
    diagram = builder.Build()
    simulator = Simulator(diagram)
    visualizer.StartRecording(False)
    simulator.AdvanceTo(x_traj.end_time())
    visualizer.PublishRecording()


def BallDrop():
    xml = """
<mujoco model="test">
<worldbody>
    <geom name="ramp1" type="box" size="50 50 50" pos="0 0 -57.735"
        euler="0 30 0" friction="5 0 0" />
    <geom name="ramp2" type="box" size="50 50 50" pos="0 0 -50" 
        friction="5 0 0" />
    <body name="body">
    <joint type="slide" name="x" axis="1 0 0" />
    <joint type="slide" name="z" axis="0 0 1" />
    <joint type="hinge" name="θ" axis="0 1 0" />
    <geom name="ball" type="sphere" size="1" mass="1"
        rgba="0.6 0.2 0.2 1" friction="5 0 0" />
    </body>
</worldbody>
</mujoco>"""

    builder = RobotDiagramBuilder()
    builder.parser().AddModelsFromString(xml, "xml")
    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    plant = diagram.plant()
    plant_context = plant.GetMyContextFromRoot(context)

    kMinTimeStep = 0.05
    kMaxTimeStep = 1.0
    hybrid = HybridMultibodyCollocation(
        diagram, context, kMinTimeStep, kMaxTimeStep
    )
    prog = hybrid.prog()
    inspector = hybrid.model_inspector()
    ball = inspector.GetGeometryIdByName(
        plant.GetBodyFrameIdOrThrow(plant.GetBodyByName("body").index()),
        Role.kProximity,
        "test::ball",
    )
    ramp1 = inspector.GetGeometryIdByName(
        inspector.world_frame_id(), Role.kProximity, "ramp1"
    )
    ramp2 = inspector.GetGeometryIdByName(
        inspector.world_frame_id(), Role.kProximity, "ramp2"
    )

    kNumTimeSteps = 5
    aerial_phase = hybrid.AddMode("aerial", kNumTimeSteps, set())
    aerial_phase.AddEqualTimeIntervalsConstraints()
    ramp1_phase = hybrid.AddModeWithInelasticImpact(
        "ramp1", kNumTimeSteps, {(ball, ramp1)}
    )
    ramp1_phase.AddEqualTimeIntervalsConstraints()
    print(ball)
    print(ramp1)
    print(ramp1_phase.in_contact())
    ramp2_phase = hybrid.AddModeWithInelasticImpact(
        "ramp2", kNumTimeSteps, {(ball, ramp2)}
    )
    ramp2_phase.AddEqualTimeIntervalsConstraints()

    x0_min = [-2, 5, 0, 0, 0, 0]
    x0_max = [-2, 5, 0, 0, 0, 0]
    prog.AddBoundingBoxConstraint(
        x0_min, x0_max, hybrid.mode(0).initial_state()
    )

    result = Solve(prog)

    if not result.is_success():
        print(result.GetInfeasibleConstraintNames(prog))

    Animate(xml, hybrid, result)

    ramp1_traj = ramp1_phase.ReconstructStateTrajectory(result)
    ramp1_force = ramp1_phase.ReconstructContactForceTrajectory(
        result, (ball, ramp1)
    )
    print(ramp1_force.vector_values(ramp1_force.get_segment_times()))
    #    ramp2_force = ramp2_phase.ReconstructContactForceTrajectory(
    #        result, (ball, ramp2)
    #    )
    #    print(ramp2_force.vector_values(ramp2_force.get_segment_times()))

    #    print(result.GetSolution(ramp1_phase.final_state().T))
    #    print(result.GetSolution(ramp2_phase.initial_state().T))

    import matplotlib.pyplot as plt

    traj = hybrid.ReconstructStateTrajectory(result)
    plt.plot(
        traj.get_segment_times(),
        traj.vector_values(traj.get_segment_times())[:6, :].T,
        ".-",
        label=["x", "z", "θ", "xdot", "zdot", "θdot"],
    )
    f_ball_ramp1_traj = hybrid.ReconstructContactForceTrajectory(
        result, (ball, ramp1)
    )
    plt.plot(
        f_ball_ramp1_traj.get_segment_times(),
        f_ball_ramp1_traj.vector_values(f_ball_ramp1_traj.get_segment_times())[
            [0, 2], :
        ].T,
        ".-",
        label=["f_x", "f_z"],
    )
    plt.legend()
    plt.show()


BallDrop()

In [ ]:
100 / np.sqrt(3)